In [1]:
import sys, os
import pandas as pd

In [ ]:
## Use this to SQL query in Elements Reporting Database for faculty publication data
## https://github.com/kglibrarian/symplecticelementssql/blob/master/Data%20Source%20Publication%20IDs%20Report%20by%20Group%20of%20Authors.md

## Upload the resulting .csv from Elements

reporter_data_path = "data/SearchResult_Export_25Jul2022_035930.csv"